<a href="https://colab.research.google.com/github/Hari25483/Backend-with-ML-Models/blob/main/voice_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.2.2-2.1ubuntu2.5).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 188 kB of archives.
After this operation, 926 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudiocpp0 amd64 19.6.0-1build1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 portaudio19-dev amd64 19.6.0-1build1 [106 kB]
Fetched 188 kB in 10s (19.1 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 122349 files and directories currently instal

In [ ]:
!pip install PyAudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyAudio: filename=PyAudio-0.2.13-cp39-cp39-linux_x86_64.whl size=68702 sha256=236971c100193f9b6c4c9189e03b643c725c2a2abc4ea56b5901847b27f9af3e
  Stored in directory: /root/.cache/pip/wheels/d5/52/ae/fa7c92aa6e9f30fc009f4f5d7caf459fad3610f0a4022cf12a
Successfully built PyAudio


In [ ]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing required libraries 
from keras.models import Sequential, Model, model_from_json
import matplotlib.pyplot as plt
import keras 
import pickle
import wave  # !pip install wave
import os
import pandas as pd
import numpy as np
import sys
import warnings
import librosa
import librosa.display
import IPython.display as ipd  # To play sound in the notebook
import pyaudio
from tensorflow.keras import optimizers
import tensorflow
from IPython.display import Audio, Javascript
from scipy.io import wavfile
from base64 import b64decode
import music21
from pydub import AudioSegment

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
opt = tensorflow.keras.optimizers.RMSprop()

# Prediction

In [ ]:
# loading json and model architecture 
json_file = open('/content/model_emotion.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/content/Emotion_Voice_Model.h5")
print("Loaded model from disk")



Loaded model from disk


In [ ]:
# Lets transform the dataset so we can apply the predictions
X, sample_rate = librosa.load('/content/OAF_base_fear.wav'
                              
                              ,duration=2.5
                              ,sr=44100
                              ,offset=0.5
                             )

sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
newdf = pd.DataFrame(data=mfccs).T
newdf

,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,-14.469676,-17.543083,-27.639011,-28.062252,-28.493858,-29.917398,-30.491682,-31.589928,-33.609974,-32.930683,...,-35.02755,-37.775723,-40.245052,-39.021458,-38.599377,-39.011715,-40.28492,-43.652988,-40.887581,-39.561699


In [ ]:
# Apply predictions
newdf= np.expand_dims(newdf, axis=2)
newpred = loaded_model.predict(newdf,batch_size=16, verbose=1)
newpred

ValueError: ignored

In [ ]:
import pandas as pd
df = pd.read_csv('/content/labels (1).csv')
from sklearn.preprocessing import OneHotEncoder
Y = df['0'].values
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()


In [ ]:
# predicting on test data.
y_pred = encoder.inverse_transform(newpred)

NameError: ignored

In [ ]:
y_pred

### API

In [ ]:
!pip install -U flask-cors
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow import keras
import pandas as pd
from flask import Flask,request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
import os
df = pd.read_csv('/content/data_path.csv')
model = keras.models.load_model('/content/my_model.h5')
app = Flask(__name__)
CORS(app)
run_with_ngrok(app)


@app.route('/')
def index():
    return jsonify("Welcome to the home page")

@app.route('/audio', methods=['POST'])
def upload_audio():
    if request.method == 'POST':
        f = request.files['audio']
        f.save(f.filename)
        print(f.filename)
        path="/content/"+str(f.filename)
        print(path)
        # path="/content/OAF_base_fear.wav"
        data_a, sample_rate = librosa.load(path)
        X = []
        feature = get_features(path)
        for ele in feature:
            X.append(ele)

        Features = pd.DataFrame(X)
        Features=Features.iloc[[0]]
        Features.head()
        pred_test=model.predict(Features)

        # Load the encoder and fit it to the labels
        from sklearn.preprocessing import OneHotEncoder
        Y = df['Emotions'].values
        # As this is a multiclass classification problem onehotencoding our Y.
        encoder = OneHotEncoder()
        Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

        # predicting on test data.
        pred_test = model.predict(Features)
        y_pred = encoder.inverse_transform(pred_test)

        print(y_pred)
              
        return jsonify(y_pred[0][0])
    else:
        return 'No audio in request'

app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://084e-34-91-77-155.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 13:05:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 13:05:27] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 13:05:28] "GET / HTTP/1.1" 200 -


OAF_base_neutral.wav
/content/OAF_base_neutral.wav
1/1 [==============================] - 0s 27ms/step


INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 13:05:55] "POST /audio HTTP/1.1" 200 -


[['sad']]
